# Решение СЛАУ при помощи инструментов numpy,scipy 

In [10]:
import numpy as np
import scipy as sp
import time
# Накидаем декоратор для определения времени выполнения программы
def timer(func):
    def wrapper(*args, **kwargs):
        # start the timer
        start_time = time.time()
        # call the decorated function
        result = func(*args, **kwargs)
        # remeasure the time
        end_time = time.time()
        # compute the elapsed time and print it
        execution_time = end_time - start_time
        print(f"Execution time: {execution_time} seconds")
        # return the result of the decorated function execution
        return result
    # return reference to the wrapper function
    return wrapper

создадим матрицу и воспользуемся встроенными модулями linalg
решим СЛАУ:
$$
\begin{cases}
3x_1 + 2x_2=12,\\
2x_1 - 3x_2 = 1.
\end{cases}
$$

1) Запишем в матричном виде
   $$
   \begin{pmatrix}
   3 & 2\\
   2 & -3
   \end{pmatrix}
   \times \begin{pmatrix} 
   x_1 \\ x_2
   \end{pmatrix}
   = \begin{pmatrix} 12 \\ 1 \end{pmatrix}
   $$

In [17]:
# Matrix of SLE 
A = np.array([
    [3, 2],
    [2, -3],
])
# vector of rigth part
b = np.array([12, 1]).reshape(2, 1)

In [18]:
# definitions decorated functions
@timer
def spsolve(A, b):
    return sp.linalg.solve(A, b)

@timer
def npsolve(A, b):
    return np.linalg.solve(A, b)

In [20]:
# Запустим просто
x = sp.linalg.solve(A, b)
print(x)
x = np.linalg.solve(A, b)
print(x)

[[2.92307692]
 [1.61538462]]
[[2.92307692]
 [1.61538462]]


In [21]:
# Запустим с подсчетом времени выполнения
print(spsolve(A, b))
print(npsolve(A, b))

Execution time: 0.0005075931549072266 seconds
[[2.92307692]
 [1.61538462]]
Execution time: 9.942054748535156e-05 seconds
[[2.92307692]
 [1.61538462]]


## Решение системы с разреженными матрицами
2) Построим разреженную матрицу __$A$__. В уранении переноса возникает двухдиагональная матрица вида:
   $$
    \begin{bmatrix}
    1 & 0 & 0 & 0 \\
    -b^{n}_1 & a^{n}_1 & 0 & 0 \\
    0 & -b^{n}_2 & a^{n}_2 & 0 \\
    0 & 0 & -b^{n}_3 & a^{n}_3 \\
    \end{bmatrix}
   $$
   Решение уравнения:
   $$
    \begin{bmatrix}
    1 & 0 & 0 & 0 \\
    -b^{n}_1 & a^{n}_1 & 0 & 0 \\
    0 & -b^{n}_2 & a^{n}_2 & 0 \\
    0 & 0 & -b^{n}_3 & a^{n}_3 \\
    \end{bmatrix}
    \times
    \begin{bmatrix} u^{n+1}_0 \\ u^{n+1}_1 \\ u^{n+1}_2 \\ u^{n+1}_3 \end{bmatrix}
    = \begin{bmatrix} u^1(t^{n+1}) \\ \xi^{n}_1 \\ \xi^{n}_2 \\ \xi^{n}_3 \end{bmatrix}
   $$
   Можно найти с использованием:
   * предложенных выше функций
   * самописного метода прогонки

In [24]:
# Зададим матрицу системы
A = np.array([
    [1, 0, 0, 0],
    [-1, 2, 0, 0],
    [0, -1, 2, 0],
    [0, 0, -1, 2]
])
# vector of rigth part
b = np.array([1, 2, 3, 4]).reshape(4, 1)
print(A)
print(b)

[[ 1  0  0  0]
 [-1  2  0  0]
 [ 0 -1  2  0]
 [ 0  0 -1  2]]
[[1]
 [2]
 [3]
 [4]]


In [25]:
# Воспользуемся описанными методами в модулях numpy, scipy
print(spsolve(A, b))
print(npsolve(A, b))

Execution time: 0.0006818771362304688 seconds
[[1.   ]
 [1.5  ]
 [2.25 ]
 [3.125]]
Execution time: 0.00011491775512695312 seconds
[[1.   ]
 [1.5  ]
 [2.25 ]
 [3.125]]


## Метод прогонки для двудиагональной матрицы
   $$
    \begin{bmatrix}
    1 & 0 & 0 & 0 \\
    -b^{n}_1 & a^{n}_1 & 0 & 0 \\
    \vdots & \cdots &  \cdots & \vdots\\
    0 & -b^{n}_2 & a^{n}_2 & 0 \\
    0 & 0 & -b^{n}_m & a^{n}_m \\
    \end{bmatrix}
    \times
    \begin{bmatrix} u^{n+1}_0 \\ u^{n+1}_1 \\ \vdots \\ u^{n+1}_{m-1} \\ u^{n+1}_m \end{bmatrix}
    = \begin{bmatrix} u^1(t^{n+1}) \\ \xi^{n}_1 \\ \vdots \\ \xi^{n}_{m-1} \\ \xi^{n}_m \end{bmatrix}
   $$

   $$
   u^{n+1}_k = \frac{b^n_ku^{n+1}_{k-1} + \xi^{n}_{k}}{a^n_{k}}, k = 1..m
   $$

In [28]:
# Напишем реализацию метода прогонки для двухдиагональной матрицы
@timer
def mysolve(A, b):
    m = len(b)
    U = np.zeros(m)
    U[0] = A[0, 0]
    for k in range(1, m):
        b_k = -A[k, k-1]
        a_k = A[k, k]
        xi_k = b[k]
        U[k] = (b_k*U[k-1]+xi_k)/a_k
    return U

print(mysolve(A, b))

Execution time: 0.0002143383026123047 seconds
[1.    1.5   2.25  3.125]
